In [80]:
#This program updates the attendance time logs. It detects what the actual shift is rendered

import time, datetime
import openpyxl
wb = openpyxl.load_workbook(r'C:\Users\v.t.flores\Documents\December 31 - January 2 timelogs_cleandataiiiii.xlsx')

sheet = wb['Sheet1']


In [81]:
#Create a new column with the concatenated date and time as values.
for i in range (2, sheet.max_row+1):
    date_str = str(sheet['C' + str(i)].value).split(" ")[0] + " "+ str(sheet['D' + str(i)].value)
    date_date = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    sheet.cell(row=i, column=8).value = date_date

In [102]:
def convTimeIn(timeIn):
    hr = timeIn.strftime('%H:%M:%S').split(":")[0]
    mint = timeIn.strftime('%H:%M:%S').split(":")[1]
    sec = timeIn.strftime('%H:%M:%S').split(":")[2]
    newTime = datetime.timedelta(hours=int(hr), minutes=int(mint), seconds=int(sec))
    
    return newTime

def nearestTimeIn(timeIn): #This fcn compares the time-in to the nearest equivalent shift
    if timeIn < datetime.time(1,30,0):
        return '_1st'
        
    else:
        _2nd = datetime.timedelta(hours=7)
        _3ET = datetime.timedelta(hours=11)
        _3rd = datetime.timedelta(hours=15)
        _1ET = datetime.timedelta(hours=19)
        _1st = datetime.timedelta(hours=23)
        _2ET = datetime.timedelta(hours=3)

        tFrom_2nd = abs(convTimeIn(timeIn)-_2nd)
        tFrom_3ET = abs(convTimeIn(timeIn)-_3ET)
        tFrom_3rd = abs(convTimeIn(timeIn)-_3rd)
        tFrom_1ET = abs(convTimeIn(timeIn)-_1ET)
        tFrom_1st = abs(convTimeIn(timeIn)-_1st)
        tFrom_2ET = abs(convTimeIn(timeIn)-_2ET)


        timeInList = [(tFrom_2nd), (tFrom_3ET), (tFrom_3rd), (tFrom_1ET), (tFrom_1st), (tFrom_2ET)]

        probShiftlist = {str(tFrom_2nd): '_2nd', str(tFrom_3ET): '_3ET', str(tFrom_3rd):'_3rd', str(tFrom_1ET): '_1ET', str(tFrom_1st):'_1st', str(tFrom_2ET):'_2ET'}

        return probShiftlist[str(sorted(timeInList)[0])]
    

In [85]:
def nearestTimeOut(timeOut): #This fcn compares the time-out to the nearest equivalent shift
    
    #list of timeouts
    _2nd = datetime.timedelta(hours=15)
    _3ET = datetime.timedelta(hours=23)
    _3rd = datetime.timedelta(hours=23)
    _1ET = datetime.timedelta(hours=7)
    _1st = datetime.timedelta(hours=7)
    _2ET = datetime.timedelta(hours=15)
    
    tFrom_2nd_out = abs(convTimeIn(timeOut)-_2nd)
    tFrom_3ET_out = abs(convTimeIn(timeOut)-_3ET)
    tFrom_3rd_out = abs(convTimeIn(timeOut)-_3rd)
    tFrom_1ET_out = abs(convTimeIn(timeOut)-_1ET)
    tFrom_1st_out = abs(convTimeIn(timeOut)-_1st)
    tFrom_2ET_out = abs(convTimeIn(timeOut)-_2ET)
    
        
    timeInList = [str(tFrom_2nd_out), str(tFrom_3ET_out), str(tFrom_3rd_out), str(tFrom_1ET_out), str(tFrom_1st_out), str(tFrom_2ET_out)]
    
    probShiftlist = {str(tFrom_2nd_out): '_2nd', str(tFrom_3ET_out): '_3ET', str(tFrom_3rd_out):'_3rd', str(tFrom_1ET_out): '_1ET', str(tFrom_1st_out):'_1st', str(tFrom_2ET_out):'_2ET'}
    
    return probShiftlist[sorted(timeInList)[0]]

In [12]:
wb.save(r'C:\Users\v.t.flores\Documents\December 31 - January 2 timelogs_trialdataclean010620ii.xlsx')

In [86]:
nearestTimeOut(sheet['D245'].value)

'_3rd'

In [87]:
sheet['D245'].value

datetime.time(23, 17, 49)

In [83]:
for in_ in range(2, sheet.max_row + 1):
    for out_ in range(2, sheet.max_row + 1):
        if sheet['E' + str(in_)].value == 'IN'and sheet['E' + str(out_)].value == 'OUT':
            if sheet['A' + str(in_)].value == sheet['A' + str(out_)].value:
                if out_>in_:
                    #computing for hours worked
                    dif = sheet['H' + str(out_)].value - sheet['H' + str(in_)].value
                    sheet['K' + str(out_)].value = dif
                
                    schd_in = nearestTimeIn(sheet['D' + str(in_)].value)
                    schd_out = nearestTimeIn(sheet['D' + str(out_)].value)

                    t_in = sheet['D' + str(in_)].value
                    t_out = sheet['D' + str(out_)].value
                    
                
                    #check if 1ET or 3rd late
                    if  schd_in == '_1ET':
                        if t_out < datetime.time(hour=7): #timeOut before 7am (meaning not 1st shift)
                            print('3 (Late)')
                            #sheet['L' + str(out_)].value = '3rd (Late)'
                        else:
                            print('1ET')
                            sheet['L' + str(out_)].value = '1ET'
                    #check if 3rd late
                    if schd_in == '_3rd':
                        if t_in > datetime.time(hour=15): #timeIn after 3pm (meaning late)
                            print('3 (Late)')
                            sheet['L' + str(out_)].value = '3 (Late)'
                        elif t_out > datetime.time(hour=3) and t_out < datetime.time(hour=7):
                            print('3OT')
                            sheet['L' + str(out_)].value = '3OT'
                        else:
                            print('3')
                            sheet['L' + str(out_)].value = '3'
                    #check if 2nd late
                    if schd_in == '_2nd':
                        if t_in > datetime.time(hour=7):  #timeIn after 7am (meaning late)
                            if t_out> datetime.time(hour=19):
                                print('2 (Late)')
                                sheet['L' + str(out_)].value = '2 (Late)'
                            else:
                                print('2 (Late)')
                                sheet['L' + str(out_)].value = '2 (Late)'
                                                
                        elif t_out > datetime.time(hour=19):
                            print('2OT')
                            sheet['L' + str(out_)].value = '2OT'
                        else:
                            print('2')
                            sheet['L' + str(out_)].value = '2'
                    #check if 1st late
                    if schd_in == '_1st':
                        if t_in > datetime.time(hour=23): #timeIn after 11pm (meaning late)
                            print('1 (Late)')
                            sheet['L' + str(out_)].value = '1 (Late)'
                        elif t_out > datetime.time(hour=11):
                            print('1OT')
                            sheet['L' + str(out_)].value = '1OT'                    
                        else:
                            print('1')
                            sheet['L' + str(out_)].value = '1'
                    #check if 3rd late
                    if schd_in == '_3ET':
                        if t_out < datetime.time(hour=23):
                            print('2 (Late)')
                            sheet['L' + str(out_)].value = '2 (Late)'
                        elif t_in < datetime.time(hour=11): 
                            if t_out > datetime.time(hour=23):
                                print('3ET')
                                sheet['L' + str(out_)].value = '3ET'
                            elif t_out > datetime.time(hour=0) and t_out < datetime.time(hour=7):
                                print('3ET')
                                sheet['L' + str(out_)].value = '3ET'
                        else:
                            print('3')
                            sheet['L' + str(out_)].value = '3'         



1
1
1
2OT
2OT
2OT
2OT
2OT
2OT
1
3ET
3ET
3
3
3
3
2
2
2
2
2
2
3
3
3
2OT
2OT
1ET
1ET
1ET
3OT
3OT
3OT
3
1
1
2
2
2
2
2
2
3
3
3
2
2
2OT
2
2OT
2OT
2OT
2
2
2
2
2
2
2
1ET
1ET
1ET
1ET
1ET
2OT
2 (Late)
1ET
3
2
2
1ET
1
2
2OT
2
2
2
3
3
3
2OT
2OT
3
3
3
2
2
3
3
1
2
2
2
2
2
1
1
2OT
1ET
2OT
3ET
3
2
2OT
2
2OT
2OT
3
1
1
1OT
1OT
3ET
3ET
3ET
1OT
3ET
3
1
1
1
1OT
1OT
2 (Late)
2 (Late)
3
3
2
3OT
3OT
3OT
1
2OT
2
2
2
2
2
1
1
3
2
3
3
1ET
2
1
2
2OT
2
2
2
1ET
3
3
2
2
2
2
2
2
2
2OT
3ET
3ET
3
2OT
2OT
2OT
3
2
2
2
2
2
2
2
2OT
2
2OT
2
2
3
3
1ET
3
2 (Late)
2
2
2
3ET
3ET
3
3
3
3
3
3ET
3ET


In [84]:
wb.save(r'C:\Users\v.t.flores\Documents\DTR_outputi.xlsx')

In [ ]:
#attempts for sheet 2

In [88]:
convTimeIn(datetime.time(23, 17, 49))

datetime.timedelta(seconds=83869)

In [90]:
datetime.time(23, 17, 49) < datetime.time(1, 23, 49)

False

In [99]:
nearestTimeIn(datetime.time(1, 30, 0))

'_3rd'

In [101]:
def nearestTimeIntest(timeIn): #This fcn compares the time-in to the nearest equivalent shift
    if timeIn < datetime.time(1,30,0):
        return '_1st'
        
    else:
        _2nd = datetime.timedelta(hours=7)
        _3ET = datetime.timedelta(hours=11)
        _3rd = datetime.timedelta(hours=15)
        _1ET = datetime.timedelta(hours=19)
        _1st = datetime.timedelta(hours=23)
        _2ET = datetime.timedelta(hours=3)

        tFrom_2nd = abs(convTimeIn(timeIn)-_2nd)
        tFrom_3ET = abs(convTimeIn(timeIn)-_3ET)
        tFrom_3rd = abs(convTimeIn(timeIn)-_3rd)
        tFrom_1ET = abs(convTimeIn(timeIn)-_1ET)
        tFrom_1st = abs(convTimeIn(timeIn)-_1st)
        tFrom_2ET = abs(convTimeIn(timeIn)-_2ET)


        timeInList = [str(tFrom_2nd), str(tFrom_3ET), str(tFrom_3rd), str(tFrom_1ET), str(tFrom_1st), str(tFrom_2ET)]

        probShiftlist = {str(tFrom_2nd): '_2nd', str(tFrom_3ET): '_3ET', str(tFrom_3rd):'_3rd', str(tFrom_1ET): '_1ET', str(tFrom_1st):'_1st', str(tFrom_2ET):'_2ET'}
        
        print(sorted(timeInList))
        return probShiftlist[sorted(timeInList)[0]]

nearestTimeIntest(datetime.time(1, 30, 0))

['13:30:00', '17:30:00', '1:30:00', '21:30:00', '5:30:00', '9:30:00']


'_3rd'

In [113]:
nearestTimeIn(datetime.time(9, 45, 0))

'_3ET'